# Initial Data Exploration
__Author:__ Gabriel Obsequio Ponón  
__Date:__ August 5, 2023

This notebook explores data collected from July 30 to August 4, on recorded vehicle coordinates, collected every second when a route is active.

This exploration looks at the structure and content of the data, along with basic descriptors and information about the distribution on each variable. This preliminary information is then used to plot the data on a map per day, as well as mapping stops and routes included in the set.

The notebook concludes with plotting the average duration of getting from one stop to the next, as well as a rough ordered list of destinations a bus would take in a given day.

See our README for more information about the project.

## Module Import

In [ ]:
import pandas as pd
import geopandas as gp # library for visualizing data on a map
import matplotlib.pyplot as plt
import contextily as cx # for creating static map plots

from tqdm import tqdm # library for displaying 

## Data Retrieval and Cleaning

### Importing and Sampling Data

In [ ]:
# Retrieve weekly data as csv
vehicle_data = pd.read_csv('data/vehicles_weekly_20230805.csv', low_memory=False)
vehicle_data

The data contains at most 570,000 samples! We collect a sample of 10,000 points to inspect and explore the data.

In [ ]:
vehicle_data["callName"].unique()

In [ ]:
vehicle_data = vehicle_data.sample(10_000)

We group the data by route ID.

In [ ]:
vehicle_data_routes = vehicle_data.groupby('routeId')

We get a shorthand description of the variables for each route.

In [ ]:
vehicle_data_routes.describe().T

We can make a few observations:

* There is only one route and one vehicle in this dataset.
* The bus position range from 4.149962 to 4.151316 degrees in latitude and -8.161963 to 8.159947 degrees longitude.
* These coordinates seem to belong to the World Mercator map system.
* Heading is between 1 to 259 degrees, which makes sense for vehicle direction.

### Stop and Route Information

Unfortunately, the OpenAPI for Transloc does not seem to offer the route ID mappings any longer. The same is true for the stop to route mappings.

We proceed with the exploration we can perform.

## Creating and Plotting Geoframe

We can convert the data into a geopandas dataframe, where the data is also indexed by a specified map geometry. We first define the function needed to generate one.

In [ ]:
def gen_gdf(df_inp): # take a regular data frame as input
    gdf_out = gp.GeoDataFrame(df_inp,
                              # convert x-y coordinates into point geometries
                              geometry=gp.points_from_xy(df_inp["long"], df_inp["lat"]),
                              # use Web Mercator Projection
                              crs="EPSG:4326")
    return gdf_out

In [ ]:
vehicle_gdf = gen_gdf(vehicle_data)
vehicle_gdf

We then preview the sampled data points on a map.

In [ ]:
fig, ax = plt.subplots(1,1, layout="tight", dpi=150)
vehicle_gdf.plot(ax=ax, color="black")
cx.add_basemap(ax, crs= "EPSG:4326")
plt.show()

We can see that our data tracks well with a regular shuttle route. However, we also observe that these routes seem to be a combination of multiple routes. 

> This could imply that our dataset no longer labels the routes correctly. This complements the fact that the stop and route information is also not available. We verify later if this is true for all collected data points.

### Data Indexing

We proceed to index our data by timestamp to prepare for further exploration.

In [ ]:
# conform date to standard format
vehicle_data["updatedAt"] = pd.to_datetime(vehicle_data["updatedAt"], format="ISO8601", utc=True)
datetime_idx = pd.DatetimeIndex(vehicle_data["updatedAt"].values).tz_localize("US/Eastern")
datetime_idx

In [ ]:
vehicle_data = vehicle_data.set_index(datetime_idx)

We then set up the data to visualize our sample for each day. We start by only keeping the coordinates and the timestamp, while dropping everything else.

In [ ]:
vehicle_data = vehicle_data.loc[:, ["lat", "long"]]
vehicle_data

We check if our data is indexing correctly

In [ ]:
vehicle_data_first = vehicle_data.loc["2023-07-30"]
vehicle_data_first.sort_index()

We observe that our data start collecting from a few minutes after midnight until midnight of that date.

We then create a geopandas dataframe to prepare for plotting.

In [ ]:
vehicle_data_gdf = gen_gdf(vehicle_data)
vehicle_data_gdf

We resample the dataframe to group into the different days constituting our timestamp.

In [ ]:
vehicle_daily = vehicle_data.resample("D")

We inspect the saved image to compare for various days

In [ ]:
# num_cols = 2
# num_rows = len(vehicle_daily) // num_cols + 1
# fig_size = 14.4 * num_cols, 9.6 * num_rows # matplotlib describes length in screen units, not px
# fig, ax = plt.subplots(num_rows, num_cols, figsize=fig_size, layout='tight')
# fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

# for i, (name, data) in enumerate(vehicle_daily):
#     col = i % num_cols
#     row = i // num_cols
#     gdf = gen_gdf(data)
#     ax[row, col].set_title(str(name))
#     gdf.plot(ax=ax[row, col], color="black")
#     cx.add_basemap(ax[row, col], crs= "EPSG:4326")

# #fig.savefig("data/weekly_sample.png")
# plt.close()

We then plot each group separately for each day.

![Weekly Summary](data/weekly_sample.png)

> An full resolution image is included in the report folder for reference.

We can make some comparisons. Throughout the week, The dates of July 30 and August 5 seem to show that the vehicle was only in one spot. This might due to a system deactivation during the weekend, or the driver forgot to turn it on.

On the other hand, the dates in between show striking similarity considering the sample was retrieved randomly (Guassian normal). However, these maps also show that the routes were probably superimposed on top of one another, preventing our further analysis at the moment.

The maps also show interesting hotspots that fall outside what looks like the bus routes. We surmise the cluster on the North-east are drivers filling up gas, while the isolated cluster in the West might be the driver's parking lots.

# Route and Stop Plots

We retrieved data provided in the public facing [CWRU shuttle tracking app](https://case.edu/access-services/transportation/shuttles/shuttle-tracking). We took note of the API used to collect the prediction information and used this data to locate stops and route paths.

## Stops

We first retrieve stop data. This data was generated from the stops attribute found in the arrival predictions data for each vehicle.

In [ ]:
stop_df = pd.read_csv("data/stop_df.csv")
stop_df = stop_df.loc[:, ["name", "lat", "long", "route_id", "route_stop_id"]]
stop_df = stop_df.set_index("name")

### Keying with coordinates

We can convert the Route to a geopandas dataframe that includes the coordinates as geometries that we can plot on a map. We start by converting the coordinates into points.

In [ ]:
stops_gdf = gen_gdf(stop_df)
stops_gdf = stops_gdf.drop(["lat", "long"], axis="columns")
stops_gdf

## Routes

We do the same with for the route data.

In [ ]:
route_df = pd.read_csv("data/vehicle_df.csv")
route_df = route_df.loc[~route_df["encoded_polyline"].isna(), ["route_name", "encoded_polyline", "color", "route_id"]]
route_df = route_df.set_index("route_name")
route_df

The __route_id__ column contains an ID format that seems to be shared with the __stops__ dataframe. This allows us to map the stops to the routes once we plot.

But first, we endeavor to convert the __encoded_polyline__ series into a format we can parse for plotting.

### Converting polylines into coordinates

The __encoded_polyline__ column contains encoded line geometries to trace the paths. We convert them as follows:

We first import the polyline module to decode the series.

In [ ]:
import polyline

decoded_points = polyline.decode(route_df.at["Blue Link", "encoded_polyline"])
print("\n".join([f"({a}, {b})" for a, b in decoded_points[:5]])) # preview first 5 points

We attempt to plot the decoded points to see how they work in action. 

In [ ]:
from shapely.geometry import LineString # import function for parsing points into lines

sample_points = [(b, a) for a, b in decoded_points]
line = LineString(sample_points)
gdf = gp.GeoDataFrame(geometry=[line])

fig, ax = plt.subplots(1,1, layout="tight")
gdf.plot(ax=ax, color=route_df.at["Blue Link", "color"]) # select color for the route
cx.add_basemap(ax, crs= "EPSG:4326")
plt.show()

> The image does not contain arrows. We will demonstrate how arrows can be added in future reports.

We then map this process to all routes. 

In [ ]:
def get_polyline(geocode_string): # define a mapping function for all geocodes
    decoded_points = polyline.decode(geocode_string)
    sample_points = [(b, a) for a, b in decoded_points]
    return LineString(sample_points)

In [ ]:
geom_polylines = route_df["encoded_polyline"].apply(get_polyline)

We can create _geodataframes_ from our routes.

In [ ]:
routes_gdf = gp.GeoDataFrame(data=route_df, geometry=geom_polylines)
routes_gdf = routes_gdf.drop("encoded_polyline", axis="columns")
routes_gdf

### Plotting Routes

We then proceed to plotting our dataframe.

In [ ]:
fig, ax = plt.subplots(1,1, layout="tight")
routes_gdf.plot(ax=ax, color=routes_gdf["color"]) # select color for the route
cx.add_basemap(ax, crs= "EPSG:4326", source=cx.providers.Stamen.TonerLite)

We are able to plot all of the routes present in the dataset. This dataset tried to include all of the routes, and missing routes can be progressively added to this dataframe. 

We then show each route individually.

In [ ]:
# num_cols = 3
# num_rows = len(routes_gdf) // num_cols + 1

# fig_size = 14.4 * num_cols, 9.6 * num_rows # matplotlib describes length in screen units, not px
# fig, ax = plt.subplots(num_rows, num_cols, figsize=fig_size, layout='tight')
# fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

# plot_gdf = routes_gdf.reset_index()

# for i in tqdm(range(len(plot_gdf))):
#     col = i % num_cols
#     row = i // num_cols
#     ax[row, col].set_title(plot_gdf.at[i, "route_name"])
#     plot_gdf.loc[[i], "geometry"].plot(ax=ax[row, col], color=plot_gdf.at[i, "color"])
    
#     try:
#         cx.add_basemap(ax[row, col], crs= "EPSG:4326")
#     except:
#         cx.add_basemap(ax[row, col], crs= "EPSG:4326", zoom=0)
# fig.savefig("data/route_sample.png")
# plt.show()
# plt.close()

> The image file is also included in the folder for better preview and zoom.
> 
We can see that some routes, namely Route B and C, are shown without proper map backgrounds. This probably has to do with the zoom level chosen for the data. The error-prone overall image generated before shows that this is probably a bug.

## Combining stops and routes

We can add routes to the stop information table to generate a list of stops to plot. We can do so by merging the stops with the routes by the route ID, deferring to the routes table.

We first inspect our data

In [ ]:
stops_gdf

In [ ]:
routes_gdf

We proceed with the merge:

In [ ]:
routes_gdf_reset = routes_gdf.reset_index()
stops_with_routes_gdf = stops_gdf.merge(routes_gdf_reset, on='route_id' , how='inner', suffixes=['_stop', '_route'])
stops_with_routes_gdf

### Plotting categorized stops

We can then plot the stops with the corresponding colors of the routes.

In [ ]:
plot_gdf = gp.GeoDataFrame(data=stops_with_routes_gdf, geometry=stops_with_routes_gdf["geometry_stop"])
plot_gdf = plot_gdf.drop(["route_id", "geometry_stop", "geometry_route"], axis="columns")
plot_gdf = plot_gdf.set_index("route_stop_id")
plot_gdf

In [ ]:
fig, ax = plt.subplots(1,1, layout="tight", dpi=150)
plot_gdf.plot(ax=ax, c=stops_with_routes_gdf["color"])
cx.add_basemap(ax, crs= "EPSG:4326")
plt.show()

We can observe that some of the routes are not present. This is especially the case when multiple routes share the same stop.

# Conclusion

This exploration shows the viability of using real time bus location data to extract information about driver behavior that is not usually revealed in consumer-facing tracking apps. The plots show that a small subset of the realtime data can be used for analysis using open-source tools.

However, the preliminary exmination of the data also show a clear lack of labelling features on the position dataset, which, from our previous experience, was not the case before the shuttle tracking upgrade. We would need to collect data directly from the consumer-facing app, albeit at less resolution, in order to label the data and continue analysis. The following section summarizes our plans moving forward.

## Future Work

1. Collect route data throughout day to retrieve complete list of routes.
2. Perform correlation analysis on route segments with collected data. The API can be retrieved from the Case Shuttle Tracking Website through the Networks console. Also ask school for access to API once back on campus.
3. Start collecting and analyzing arrivals data and correlated with original dataset.